In [ ]:
# set path
path = "C:/Users/OhRai/Desktop/" # path containing CUB_200_2011

# libraries
from PIL import Image
import os
import pandas as pd
import Augmentor
import time
import torch
import numpy as np

# functions
def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
# process data

# raw data path
makedir(path + 'ProtoPNet-master/datasets/cub200_cropped/test_cropped/')
makedir(path + 'ProtoPNet-master/datasets/cub200_cropped/train_cropped/')

# join data in single dataframe
df_image = pd.read_csv(path + "CUB_200_2011/images.txt", delimiter = " ", names = ["id", "path"])
df_bounding = pd.read_csv(path + "CUB_200_2011/bounding_boxes.txt", delimiter = " ", names = ["id", "x", "y", "w", "h"])
df_train_test_split = pd.read_csv(path + "CUB_200_2011/train_test_split.txt", delimiter = " ", names = ["id","split"])
df = pd.merge(pd.merge(df_image, df_bounding) , df_train_test_split)

# crop and save images
for index, row in df.iterrows():
    id, x, y, w, h, split = str(row["id"]), float(row["x"]), float(row["y"]), float(row["w"]), float(row["h"]), float(row["split"]) # 1 train 0 test
    img_path = str(row["path"])
    im = Image.open(path + "CUB_200_2011/images/" + row["path"])
    img_dir, img_name = img_path.split("/")
    img = im.crop((x, y, x + w, y + h))
    if split == 0:
        makedir(path + 'ProtoPNet-master/datasets/cub200_cropped/test_cropped/' + img_dir)
        img.save(path + 'ProtoPNet-master/datasets/cub200_cropped/test_cropped/' + img_path[:-4] + '.jpg', 'JPEG')
    else:
        makedir(path + 'ProtoPNet-master/datasets/cub200_cropped/train_cropped/' + img_dir)
        img.save(path + 'ProtoPNet-master/datasets/cub200_cropped/train_cropped/' + img_path[:-4] + '.jpg', 'JPEG')

# augment data

# setup path
datasets_root_dir = path + 'ProtoPNet-master/datasets/cub200_cropped/'
dir = datasets_root_dir + 'train_cropped/'
target_dir = datasets_root_dir + 'train_cropped_augmented/'
makedir(target_dir)
folders = [os.path.join(dir, folder) for folder in next(os.walk(dir))[1]]
target_folders = [os.path.join(target_dir, folder) for folder in next(os.walk(dir))[1]]

# data augmentation
for i in range(len(folders)):
    fd = folders[i]
    tfd = target_folders[i]
    # rotation
    p = Augmentor.Pipeline(source_directory=fd, output_directory=tfd)
    p.rotate(probability=1, max_left_rotation=15, max_right_rotation=15)
    p.flip_left_right(probability=0.5)
    for i in range(10):
        p.process()
    del p
    # skew
    p = Augmentor.Pipeline(source_directory=fd, output_directory=tfd)
    p.skew(probability=1, magnitude=0.2)  # max 45 degrees
    p.flip_left_right(probability=0.5)
    for i in range(10):
        p.process()
    del p
    # shear
    p = Augmentor.Pipeline(source_directory=fd, output_directory=tfd)
    p.shear(probability=1, max_shear_left=10, max_shear_right=10)
    p.flip_left_right(probability=0.5)
    for i in range(10):
        p.process()
    del p


In [ ]:
base_architecture = 'vgg19'
img_size = 224
prototype_shape = (2000, 128, 1, 1)
num_classes = 200
prototype_activation_function = 'log'
add_on_layers_type = 'regular'

experiment_run = '003'

data_path = path + 'ProtoPNet-master/datasets/cub200_cropped/'
train_dir = data_path + 'train_cropped_augmented/'
test_dir = data_path + 'test_cropped/'
train_push_dir = data_path + 'train_cropped/'
train_batch_size = 80
test_batch_size = 100
train_push_batch_size = 75

joint_optimizer_lrs = {'features': 1e-4,
                       'add_on_layers': 3e-3,
                       'prototype_vectors': 3e-3}
joint_lr_step_size = 5

warm_optimizer_lrs = {'add_on_layers': 3e-3,
                      'prototype_vectors': 3e-3}

last_layer_optimizer_lr = 1e-4

coefs = {
    'crs_ent': 1,
    'clst': 0.8,
    'sep': -0.08,
    'l1': 1e-4,
}

num_train_epochs = 1000
num_warm_epochs = 5

push_start = 10
push_epochs = [i for i in range(num_train_epochs) if i % 10 == 0]
